In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, floor
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler, StringIndexerModel, OneHotEncoderModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.sql.types import DoubleType
from itertools import product

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder\
    .appName("Classification RandomForest")\
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/21 09:57:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/21 09:57:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


DataFrame[]

In [3]:
# Charger les données de la table marketing
df_marketing = spark.sql("SELECT * FROM marketing")
df_marketing.printSchema()

df_marketing.show()

# Charger les modèles de transformation
indexer_model = StringIndexerModel.load("hdfs://namenode:9000/user/model/indexer_model")
indexer_sexe = StringIndexerModel.load("hdfs://namenode:9000/user/model/indexer_sexe")
indexer_situationfamiliale = StringIndexerModel.load("hdfs://namenode:9000/user/model/indexer_situationfamiliale")
encoder_sexe = OneHotEncoderModel.load("hdfs://namenode:9000/user/model/encoder_sexe")
encoder_situationfamiliale = OneHotEncoderModel.load("hdfs://namenode:9000/user/model/encoder_situationfamiliale")

# Appliquer les transformations nécessaires (identiques à celles de l'entraînement)
df_marketing = indexer_sexe.transform(df_marketing)
df_marketing = encoder_sexe.transform(df_marketing)
df_marketing = indexer_situationfamiliale.transform(df_marketing)
df_marketing = encoder_situationfamiliale.transform(df_marketing)

# Transformer les colonnes booléennes en entiers
df_marketing = df_marketing.withColumn(
    "deuxiemevoiture",
    when(col("deuxiemevoiture") == False, 0)
    .when(col("deuxiemevoiture") == True, 1)
    .otherwise(col("deuxiemevoiture").cast("int"))
)

df_marketing = df_marketing.withColumn(
    "taux_eligible",
    when(col("taux_eligible") == False, 0)
    .when(col("taux_eligible") == True, 1)
    .otherwise(col("taux_eligible").cast("int"))
)

# Définir les colonnes utilisées pour les features
feature_cols = [col for col in df_marketing.columns if col not in ["categorie", "label", 'sexe', 'situationfamiliale']]
df_marketing.show()


# Assembler les caractéristiques en un vecteur de features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
prediction_data = assembler.transform(df_marketing).select("features")

# Charger le modèle depuis HDFS
hdfs_path = "hdfs://namenode:9000/user/model/categorie"
model = RandomForestClassificationModel.load(hdfs_path)

# Effectuer les prédictions
marketingPrediction = model.transform(prediction_data)

# Mapper les prédictions avec les catégories d'origine
label_to_category = IndexToString(
    inputCol="prediction",
    outputCol="predicted_category",
    labels=indexer_model.labels
)
result = label_to_category.transform(marketingPrediction)
result.printSchema()
# Afficher les résultats
result.select("features", "prediction", "predicted_category").show()

# Arrêter la SparkSession
spark.stop()

root
 |-- age: integer (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: float (nullable = true)
 |-- situationfamiliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: boolean (nullable = true)
 |-- taux_eligible: boolean (nullable = true)



+---+----+------+------------------+---------------+---------------+-------------+
|age|sexe|  taux|situationfamiliale|nbenfantacharge|deuxiemevoiture|taux_eligible|
+---+----+------+------------------+---------------+---------------+-------------+
| 21|   F|1396.0|       celibataire|              0|          false|        false|
| 35|   M| 223.0|       celibataire|              0|          false|         true|
| 48|   M| 401.0|       celibataire|              0|          false|         true|
| 26|   F| 420.0|         en couple|              3|           true|         true|
| 80|   M| 530.0|         en couple|              3|          false|         true|
| 27|   F| 153.0|         en couple|              2|          false|         true|
| 59|   F| 572.0|         en couple|              2|          false|        false|
| 43|   F| 431.0|       celibataire|              0|          false|         true|
| 64|   M| 559.0|       celibataire|              0|          false|        false|
| 22